# Суть задания


1.   Реализовать решение задачи верификации лица (сравнение 1 к 1)
2.   2 на кого из МИЭМ вы похожи + 2 GUI
2.   Протестировать работу различных методов трекинга объектов в различных задачах.

Примерное время выполнения - 8-10 часов.

Дедлайн: 1 февраля в 23:59

Штрафные коэффициенты оценивания при просрочке:

-|-
----|---
Просрочка меньше 12 часов | без штрафа
От 12 часов до 7 дней после дедлайна | 0.8
От 7 до 14 дней | 0.6
Более 14 дней и до начала сессии | 0.4

## Оценивание

Задание:

Часть работы | Стоимость в баллах
-------------|--------------------
Детекция + Верификация **Можно сдать без защиты**.| 3
Поисковик "MIEM Lookalikes" **Можно сдать без защиты**.| 2
MIEM Lookalike Web-GUI **Можно сдать без защиты**.| 1
Исследование устойчивости методов распознавания | 4
Итого | 10 баллов

Формула оценивания всей работы:

О = Задание * 0.9 + Тест на лекции * 0.1



## Часть 2. Оценить точность работы методов из DeepFace на аугментированных данных (2 балла).

Необходимо собрать собственный набор данных из **различных** изображений Вашего лица с разных ракурсов, желательно настоящие фотографии из личного архива (20 штук)\
Возьмите эталонное изображение (как в паспорте) и при помощи библиотеки [DeepFace](https://github.com/serengil/deepface) проверьте его на соответствие всему датасету. Посчитайте метрику Precision. \
\
Примените каждую из перечисленных ниже аугментаций (**по-отдельности**) ко всему датасету и измерьте метрику Precision для измененнного датасета:
*   Поворот изображения на 45° и 90°.
*   Добавление шума (Gaussian Noise).
*   Изменение яркости (увеличение и уменьшение на 50%).
*   Размытие с различными параметрами.
\
Реузультаты соберите в таблицу вида:

Метод | Исходный датасет | Поворот на 45° | Поворот на 90° | Изображение с шумом |
--- | ----|--- | --- | --- |
VGG-Face | 0 | 0 | 0 | 0 |
Facenet | 0 | 0 | 0 | 0 |
Facenet512 | 0 | 0 | 0 | 0 |
OpenFace | 0 | 0 | 0 | 0 |
DeepFace | 0 | 0 | 0 | 0 |
DeepID | 0 | 0 | 0 | 0 |
ArcFace | 0 | 0 | 0 | 0 |
Dlib | 0 | 0 | 0 | 0 |
SFace | 0 | 0 | 0 | 0 |
GhostFaceNet | 0 | 0 | 0 | 0 |

In [102]:
import cv2
import numpy as np
import pandas as pd
import os
from deepface import DeepFace
from sklearn.metrics import precision_score
from glob import glob

In [103]:
def augmentations(image, aug):

    if aug == "rotate_45":
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        matrix = cv2.getRotationMatrix2D(center, 45, 1.0)
        return cv2.warpAffine(image, matrix, (w, h))
    
    elif aug == "rotate_90":
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    elif aug == "gaussian_noise":
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        return cv2.add(image, noise)
    
    elif aug == "brightness_up":
        return cv2.convertScaleAbs(image, alpha=1.5, beta=0)
    
    elif aug == "brightness_down":
        return cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    
    elif aug == "blur":
        return cv2.GaussianBlur(image, (5, 5), 0)
    
    else:
        return image

In [114]:
def print_metric(model_name, reference_img, dataset):
    y_true, y_pred = [], []
    corr = 0

    for img in dataset:
        
        try:
            result = DeepFace.verify(reference_img, img, model_name=model_name, enforce_detection=False)
            if result['verified']:
              corr += 1
              
        except:
            pass

    return corr / len(dataset) if len(dataset) != 0 else 0

In [115]:
def process_dataset(reference_img, dataset_path):
    augs = ["original", "rotate_45", "rotate_90", "gaussian_noise", "brightness_up", "brightness_down", "blur"]
    models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace", "GhostFaceNet"]
    res = {aug: [] for aug in augs}

    images = glob(os.path.join(dataset_path, "*"))

    original_images = [cv2.imread(img) for img in images]

    augmented_datasets = {aug: [augmentations(img, aug) for img in original_images] for aug in augs}

    for model in tqdm(models, total = len(models)):

        for aug in tqdm(augs, total = len(augs)):

            augmented_images_paths = []
            for i, img in enumerate(augmented_datasets[aug]):
                
                aug_path = f"temp_{aug}_{i}.jpg"
                cv2.imwrite(aug_path, img)
                augmented_images_paths.append(aug_path)

            precision = print_metric(model, reference_img, augmented_images_paths)
            res[aug].append(precision)

            for path in augmented_images_paths:
                os.remove(path)

    df = pd.DataFrame(res, index=models)
    print(df)
    df.to_csv("results.csv")

In [116]:
reference_image_path = "me.jpg"
dataset_directory = "drive/MyDrive/IMMM_25/my_photos/"
process_dataset(reference_image_path, dataset_directory)

100%|██████████| 10/10 [1:10:19<00:00, 421.97s/it]

              original  rotate_45  rotate_90  gaussian_noise  brightness_up  \
VGG-Face      0.619048   0.238095   0.000000        0.285714       0.666667   
Facenet       0.571429   0.142857   0.000000        0.000000       0.666667   
Facenet512    0.571429   0.190476   0.000000        0.000000       0.619048   
OpenFace      0.047619   0.000000   0.000000        0.000000       0.095238   
DeepFace      0.000000   0.000000   0.000000        0.000000       0.000000   
DeepID        0.190476   0.095238   0.047619        0.238095       0.333333   
ArcFace       0.619048   0.142857   0.000000        0.000000       0.666667   
Dlib          0.000000   0.000000   0.000000        0.000000       0.000000   
SFace         0.619048   0.047619   0.000000        0.000000       0.571429   
GhostFaceNet  0.571429   0.095238   0.000000        0.000000       0.571429   

              brightness_down      blur  
VGG-Face             0.714286  0.714286  
Facenet              0.333333  0.571429  
Face

# Итоговая проверка и отчётность



## Задание 1
* Ссылка на исходный код приложения в GitHub
* Ссылка на видеозапись экрана ПК во время работы приложения

## Задание 2
* Ссылка на исходный код приложения в GitHub
* Ссылка на видеозапись экрана ПК во время работы приложения

## Задание 3
* Ссылка на Jupyter Notebook с кодом подсчёта метрики по кадрам с видео в GitHub
* Ссылка на архив в формате ZIP или 7z с вашими фото
* Ссылка на Jupyter Notebook с кодом наложения аугментаций и подсчёта метрики в GitHub

# Контрольные вопросы
